In [2]:
# import module
import pandas as pd
import numpy as np
import gurobipy as gp

In [3]:
# import csv
s2019 = pd.read_csv('./stocks2019.csv', index_col=0)
s2020 = pd.read_csv('./stocks2020.csv', index_col=0)
s2019.head()

,NDX,ATVI,ADBE,AMD,ALXN,ALGN,GOOGL,GOOG,AMZN,AMGN,...,TCOM,ULTA,VRSN,VRSK,VRTX,WBA,WDAY,WDC,XEL,XLNX
X,,,,,,,,,,,,,,,,,,,,,
2019-01-02,6360.870117,46.350380,224.570007,18.830000,98.050003,202.119995,1054.680054,1045.849976,1539.130005,182.458298,...,27.590000,247.970001,147.759995,107.364159,164.080002,63.920757,159.740005,36.312473,46.041862,84.600266
2019-01-03,6147.129883,44.704514,215.699997,17.049999,100.209999,184.779999,1025.469971,1016.059998,1500.280029,179.681961,...,26.959999,243.360001,142.589996,104.083450,163.729996,63.253742,154.020004,33.835327,45.860447,81.414970
2019-01-04,6422.669922,46.488358,226.190002,19.000000,106.000000,186.710007,1078.069946,1070.709961,1575.390015,185.824142,...,28.549999,255.029999,148.970001,108.678413,172.699997,65.358139,163.350006,35.268467,46.309223,85.184715
2019-01-07,6488.250000,47.799141,229.259995,20.570000,107.940002,189.919998,1075.920044,1068.390015,1629.510010,188.324738,...,29.180000,271.000000,151.399994,108.787117,175.850006,65.733910,163.500000,36.521275,46.108707,87.434891
2019-01-08,6551.850098,49.247898,232.679993,20.750000,108.610001,192.949997,1085.369995,1076.280029,1656.579956,190.739777,...,29.480000,276.000000,156.919998,109.755516,178.190002,67.049149,166.949997,36.170105,46.643410,85.769180


In [4]:
# Create a new data frame for correlation matrix
s2019_NDX = s2019['NDX'].copy()
s2019_stock = s2019.copy().drop('NDX', axis=1)
s2020_NDX = s2020['NDX'].copy()
s2020_stock = s2020.copy().drop('NDX', axis=1)
s2020_stock.head()

,ATVI,ADBE,AMD,ALXN,ALGN,GOOGL,GOOG,AMZN,AMGN,ADI,...,TCOM,ULTA,VRSN,VRSK,VRTX,WBA,WDAY,WDC,XEL,XLNX
1/2/20,58.266792,334.429993,49.099998,107.839996,283.679993,1368.680054,1367.369995,1898.010010,231.917572,117.996941,...,36.970001,254.550003,196.729996,150.167465,219.449997,56.046688,167.460007,64.771545,60.782249,100.115349
1/3/20,58.286655,331.809998,48.599998,106.410004,280.440002,1361.520020,1360.660034,1874.969971,230.343124,115.919769,...,36.180000,250.169998,200.880005,151.628220,217.979996,56.046688,168.440002,63.774597,61.074562,97.810677
1/6/20,59.349670,333.709991,48.389999,106.580002,285.880005,1397.810059,1394.209961,1902.880005,232.110748,114.557854,...,35.689999,250.949997,202.740005,151.976013,224.029999,56.530506,169.490005,62.550629,60.986866,95.771927
1/7/20,59.945747,333.390015,48.250000,106.849998,283.059998,1395.109985,1393.339966,1906.859985,229.927765,117.164116,...,37.330002,253.089996,203.210007,153.247955,223.789993,56.245903,172.949997,66.785164,60.860199,97.958405
1/8/20,59.488754,337.869995,47.830002,108.580002,286.000000,1405.040039,1404.319946,1891.969971,230.101654,118.222298,...,36.869999,258.000000,204.149994,154.658997,231.089996,52.963551,178.710007,67.752487,60.801735,97.682640


In [5]:
# Calculate daily return
for col in s2019_stock.columns:
    s2019_stock[col] = s2019_stock[col].pct_change(1)
for col in s2020_stock.columns:
    s2020_stock[col] = s2020_stock[col].pct_change(1)

s2019_stock.drop(index='2019-01-02', inplace=True)
s2020_stock.drop(index='1/2/20', inplace=True)

s2019_NDX = s2019_NDX.pct_change(1)
s2020_NDX = s2020_NDX.pct_change(1)

# Transform NDX to numpy array
s2019_NDX = s2019_NDX.dropna()
s2019_NDX = s2019_NDX.to_numpy()

s2020_NDX = s2020_NDX.dropna()
s2020_NDX = s2020_NDX.to_numpy()


s2019_stock.head()

,ATVI,ADBE,AMD,ALXN,ALGN,GOOGL,GOOG,AMZN,AMGN,ADI,...,TCOM,ULTA,VRSN,VRSK,VRTX,WBA,WDAY,WDC,XEL,XLNX
X,,,,,,,,,,,,,,,,,,,,,
2019-01-03,-0.035509,-0.039498,-0.094530,0.022030,-0.085791,-0.027696,-0.028484,-0.025242,-0.015216,-0.060405,...,-0.022834,-0.018591,-0.034989,-0.030557,-0.002133,-0.010435,-0.035808,-0.068217,-0.003940,-0.037651
2019-01-04,0.039903,0.048632,0.114370,0.057779,0.010445,0.051294,0.053786,0.050064,0.034184,0.024278,...,0.058976,0.047954,0.044744,0.044147,0.054785,0.033269,0.060577,0.042356,0.009786,0.046303
2019-01-07,0.028196,0.013573,0.082632,0.018302,0.017192,-0.001994,-0.002167,0.034353,0.013457,0.006288,...,0.022067,0.062620,0.016312,0.001000,0.018240,0.005749,0.000918,0.035522,-0.004330,0.026415
2019-01-08,0.030309,0.014918,0.008751,0.006207,0.015954,0.008783,0.007385,0.016612,0.012824,0.024396,...,0.010281,0.018450,0.036460,0.008902,0.013307,0.020009,0.021101,-0.009615,0.011597,-0.019051
2019-01-09,0.017210,0.011819,-0.026988,0.012430,0.038196,-0.003427,-0.001505,0.001714,-0.001196,0.025340,...,0.023745,0.018804,-0.008157,0.003781,0.023065,0.012050,-0.004612,0.044083,-0.007984,0.008972


In [158]:
# Take a look at the correlation matrix
s2019_stock.corr()

,ATVI,ADBE,AMD,ALXN,ALGN,GOOGL,GOOG,AMZN,AMGN,ADI,...,TCOM,ULTA,VRSN,VRSK,VRTX,WBA,WDAY,WDC,XEL,XLNX
ATVI,1.000000,0.399939,0.365376,0.223162,0.216280,0.433097,0.426777,0.467076,0.203956,0.329355,...,0.322906,0.128241,0.464850,0.316549,0.259679,0.218149,0.311659,0.303077,0.043389,0.249667
ADBE,0.399939,1.000000,0.452848,0.368928,0.363370,0.552125,0.540404,0.598237,0.291978,0.473815,...,0.360392,0.201151,0.711339,0.541243,0.402171,0.228106,0.650430,0.361516,0.207403,0.289497
AMD,0.365376,0.452848,1.000000,0.301831,0.344252,0.418861,0.417254,0.549302,0.151452,0.503733,...,0.332776,0.210623,0.498342,0.330900,0.272983,0.281950,0.407626,0.438892,0.017283,0.478010
ALXN,0.223162,0.368928,0.301831,1.000000,0.332433,0.315993,0.307698,0.363170,0.342022,0.317040,...,0.257143,0.408936,0.350581,0.191489,0.522423,0.192720,0.416396,0.289908,0.047947,0.200356
ALGN,0.216280,0.363370,0.344252,0.332433,1.000000,0.248747,0.250316,0.399281,0.264599,0.328280,...,0.175957,0.128559,0.360886,0.251855,0.334978,0.219595,0.308968,0.284407,0.088059,0.253934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WBA,0.218149,0.228106,0.281950,0.192720,0.219595,0.232900,0.230603,0.288168,0.194490,0.347861,...,0.257049,0.145330,0.195475,0.126118,0.214585,1.000000,0.211641,0.296309,-0.130798,0.276643
WDAY,0.311659,0.650430,0.407626,0.416396,0.308968,0.379493,0.371826,0.424748,0.211712,0.351734,...,0.235045,0.269545,0.569672,0.401476,0.376047,0.211641,1.000000,0.315699,0.209240,0.254117
WDC,0.303077,0.361516,0.438892,0.289908,0.284407,0.328619,0.322110,0.419620,0.172623,0.602935,...,0.377215,0.126463,0.331916,0.308558,0.314773,0.296309,0.315699,1.000000,-0.076115,0.516958
XEL,0.043389,0.207403,0.017283,0.047947,0.088059,0.059930,0.052570,0.076724,0.137857,-0.047259,...,-0.172752,0.074686,0.280371,0.407770,0.191667,-0.130798,0.209240,-0.076115,1.000000,-0.117552


In [6]:
# Create object 
simi_list = []
for row in range(s2019_stock.corr().values.shape[0]):
    item_list = s2019_stock.corr().values[row]
    for item in item_list:
        simi_list.append(item)

In [249]:
# Set up optimization problem
m = 5

obj = np.array(simi_list+[0]*100)
var_type = [['B'] * len(obj)]

A = np.zeros((10101, len(obj)))
b = np.array([0] * 10101)

sense = np.array([''] * 10101)

# ST1: only pick m stocks
A[0, -100:] = 1
b[0] = m
sense[0] = '='

# ST2: Xij - yj <=0 
for row in range(1,10001):
    A[row, row-1] = 1
    A[row, 10000+(row-1)%100] = -1
    b[row] = 0
    sense[row] = '<'

# ST3: sum Xij = 1
for row in range(10001, 10101):
    col_start = ((row-1)%100)*100
    A[row, col_start:col_start+100] = 1
    b[row] = 1
    sense[row] = '='

In [250]:
# run optimization model
stock2019Mod = gp.Model()
stock2019Mod_x = stock2019Mod.addMVar(len(obj),vtype=['B']*len(obj)) # 'C' or 'I' or 'B'
stock2019Mod_con = stock2019Mod.addMConstrs(A, stock2019Mod_x, sense, b)
stock2019Mod.setMObjective(None,obj,0,sense=gp.GRB.MAXIMIZE)

stock2019Mod.Params.OutputFlag = 0 # tell gurobi to shut up!!
stock2019Mod.optimize()

In [251]:
stock2019Mod.objVal

54.83990652229107

In [252]:
for i in range(10000,10100):
    if stock2019Mod_x.x[i] == 1:
        print(i)

10056
10059
10063
10094
10098


In [253]:
# Calculate Portoflio Weights

# get stock returns
ret_2019 = s2019_stock.values[:,[56, 59, 63, 94, 98]]
ret_2019.shape

(250, 5)

In [254]:
# Set up weight objective
obj_weight = np.array([1]*ret_2019.shape[0] + [0]*m)

A_weight = np.zeros((2*250+1 , len(obj_weight)))

b_weight = np.array(list(s2019_NDX)+list(-s2019_NDX)+[1])
sense_weight = np.array(['>'] *(2*ret_2019.shape[0]) + ['='])

for day in range(ret_2019.shape[0]):
    A_weight[day, day] = 1
    A_weight[day, -5:] = ret_2019[day]
    
    A_weight[250+day, day] = 1
    A_weight[250+day, -5:] = -ret_2019[day]
A_weight[500, -5:] = 1

In [255]:
weightMod = gp.Model()
weightMod_x = weightMod.addMVar(len(obj_weight)) # 'C' or 'I' or 'B'
weightMod_con = weightMod.addMConstrs(A_weight, weightMod_x, sense_weight, b_weight)
weightMod.setMObjective(None,obj_weight,0,sense=gp.GRB.MINIMIZE)

weightMod.Params.OutputFlag = 0 # tell gurobi to shut up!!
weightMod.optimize()

In [219]:
weightMod.objval

0.7891782824631473

In [221]:
weightMod_x.x[-5:]

array([0.04886175, 0.21038806, 0.58035198, 0.07119022, 0.089208  ])

In [257]:
# Store the weight
weight_2019 = weightMod_x.x[-5:]

In [258]:
# Calculate 2020 difference
ret_2020 = s2020_stock.values[:,[56, 59, 63, 94, 98]]
diff_2020 = sum(abs(s2020_NDX - ret_2020 @ weight_2019))
diff_2019 = sum(abs(s2019_NDX - ret_2019 @ weight_2019))

print("For 2020, the absolute difference is {}.".format(diff_2020))
print("For 2019, the absolute difference is {}.".format(diff_2019))

For 2020, the absolute difference is 1.1124373455076466.
For 2019, the absolute difference is 0.7891782824631471.


In [287]:
m_list=[5,10,20,30,40,50,60,70,80,90]
result_2019 = []
result_2020 = []

for m in m_list:
    obj = np.array(simi_list+[0]*100)
    var_type = [['B'] * len(obj)]

    A = np.zeros((10101, len(obj)))
    b = np.array([0] * 10101)

    sense = np.array([''] * 10101)

    # ST1: only pick m stocks
    A[0, -100:] = 1
    b[0] = m
    sense[0] = '='

    # ST2: Xij - yj <=0 
    for row in range(1,10001):
        A[row, row-1] = 1
        A[row, 10000+(row-1)%100] = -1
        b[row] = 0
        sense[row] = '<'

    # ST3: sum Xij = 1
    for row in range(10001, 10101):
        col_start = ((row-1)%100)*100
        A[row, col_start:col_start+100] = 1
        b[row] = 1
        sense[row] = '='

    # run optimization model
    stock2019Mod = gp.Model()
    stock2019Mod_x = stock2019Mod.addMVar(len(obj),vtype=['B']*len(obj)) # 'C' or 'I' or 'B'
    stock2019Mod_con = stock2019Mod.addMConstrs(A, stock2019Mod_x, sense, b)
    stock2019Mod.setMObjective(None,obj,0,sense=gp.GRB.MAXIMIZE)

    stock2019Mod.Params.OutputFlag = 0 # tell gurobi to shut up!!
    stock2019Mod.optimize()

    stock_list = []
    for i in range(10000,10100):
        if stock2019Mod_x.x[i] == 1:
            stock_list.append(i-10000)
    # Calculate Portoflio Weights

    # get stock returns
    ret_2019 = s2019_stock.values[:,stock_list]

    # Set up weight objective
    obj_weight = np.array([1]*ret_2019.shape[0] + [0]*m)

    A_weight = np.zeros((2*250+1 , len(obj_weight)))

    b_weight = np.array(list(s2019_NDX)+list(-s2019_NDX)+[1])
    sense_weight = np.array(['>'] *(2*ret_2019.shape[0]) + ['='])

    for day in range(ret_2019.shape[0]):
        A_weight[day, day] = 1
        A_weight[day, -m:] = ret_2019[day]

        A_weight[250+day, day] = 1
        A_weight[250+day, -m:] = -ret_2019[day]
    A_weight[500, -m:] = 1

    weightMod = gp.Model()
    weightMod_x = weightMod.addMVar(len(obj_weight)) # 'C' or 'I' or 'B'
    weightMod_con = weightMod.addMConstrs(A_weight, weightMod_x, sense_weight, b_weight)
    weightMod.setMObjective(None,obj_weight,0,sense=gp.GRB.MINIMIZE)

    weightMod.Params.OutputFlag = 0 # tell gurobi to shut up!!
    weightMod.optimize()

    # Store the weight
    weight_2019 = weightMod_x.x[-m:]

    # Calculate 2020 difference
    ret_2020 = s2020_stock.values[:,stock_list]
    diff_2020 = sum(abs(s2020_NDX - ret_2020 @ weight_2019))
    diff_2019 = sum(abs(s2019_NDX - ret_2019 @ weight_2019))
    result_2019.append(diff_2019)
    result_2020.append(diff_2020)

In [291]:
dict= {
    'm': m_list,
    '2019_result':result_2019,
    '2020_result':result_2020
    
}

pd.DataFrame(dict)

,m,2019_result,2020_result
0,5,0.789178,1.112437
1,10,0.701218,1.102404
2,20,0.466268,0.855446
3,30,0.407021,0.756002
4,40,0.368247,0.762766
5,50,0.332540,0.772100
6,60,0.349468,1.149781
7,70,0.171729,0.537039
8,80,0.148219,0.545540
9,90,0.053779,0.367790


In [7]:
# Problem 4
run_time = 3600
ret_total_2019 = s2019_stock.values

In [21]:
# Set up the object and A,b matrices
M=1
m = 5 # number of stocks to pick
MIP_obj = np.array([1]*ret_total_2019.shape[0]+[0]*100+[0]*100)
var_type = ['C']*ret_total_2019.shape[0]+['B']*100 +['C']*100
MIP_A = np.zeros((2*ret_total_2019.shape[0]+102, len(MIP_obj)))
MIP_b = np.array(list(s2019_NDX)+list(-s2019_NDX)+[0]*100+[1,m])
MIP_sense = np.array(['>']*2*ret_total_2019.shape[0]+['<']*100+['=','='])

for day in range(ret_total_2019.shape[0]):
    MIP_A[day, day] = 1
    MIP_A[day, -100:] = ret_total_2019[day]

    MIP_A[250+day, day] = 1
    MIP_A[250+day, -100:] = -ret_total_2019[day]

for i in range(100):
    MIP_A[500+i, [250+i, 350+i]] = [-M, 1]

MIP_A[-1,ret_total_2019.shape[0]:ret_total_2019.shape[0]+100] = 1
MIP_A[-2, -100:] = 1

In [22]:
weightMod = gp.Model()
weightMod_x = weightMod.addMVar(len(MIP_obj), vtype=var_type) # 'C' or 'I' or 'B'
weightMod_con = weightMod.addMConstrs(MIP_A, weightMod_x, MIP_sense, MIP_b)
weightMod.setMObjective(None,MIP_obj,0,sense=gp.GRB.MINIMIZE)
weightMod.setParam('TimeLimit', 60)

weightMod.Params.OutputFlag = 0 # tell gurobi to shut up!!
weightMod.optimize()

Changed value of parameter TimeLimit to 60.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf


In [23]:
weightMod.objVal

0.5105420480962296

In [25]:
weight_2019 = weightMod_x.x[-100:]

# Calculate 2020 difference
ret_total_2020 = s2020_stock.values
diff_2020 = sum(abs(s2020_NDX - ret_total_2020 @ weight_2019))
diff_2019 = sum(abs(s2019_NDX - ret_total_2019 @ weight_2019))

print("2019 has a difference of {}".format(diff_2019))
print("2020 has a difference of {}".format(diff_2020))

2019 has a difference of 0.5105420480962297
2020 has a difference of 0.9440188339033769


In [8]:
# Question 5
run_time = 3600
m_list=[5,10,20,30,40,50,60,70,80,90]
MIP_result_2019 = []
MIP_result_2020 = []
MIP_abs_2019 = []
MIP_abs_2020 = []
weight_matrix = []

for m in m_list:
# Set up the object and A,b matrices
    M=1
    MIP_obj = np.array([1]*ret_total_2019.shape[0]+[0]*100+[0]*100)
    var_type = ['C']*ret_total_2019.shape[0]+['B']*100 +['C']*100
    MIP_A = np.zeros((2*ret_total_2019.shape[0]+102, len(MIP_obj)))
    MIP_b = np.array(list(s2019_NDX)+list(-s2019_NDX)+[0]*100+[1,m])
    MIP_sense = np.array(['>']*2*ret_total_2019.shape[0]+['<']*100+['=','='])

    for day in range(ret_total_2019.shape[0]):
        MIP_A[day, day] = 1
        MIP_A[day, -100:] = ret_total_2019[day]

        MIP_A[250+day, day] = 1
        MIP_A[250+day, -100:] = -ret_total_2019[day]

    for i in range(100):
        MIP_A[500+i, [250+i, 350+i]] = [-M, 1]

    MIP_A[-1,ret_total_2019.shape[0]:ret_total_2019.shape[0]+100] = 1
    MIP_A[-2, -100:] = 1
    
    weightMod = gp.Model()
    weightMod_x = weightMod.addMVar(len(MIP_obj), vtype=var_type) # 'C' or 'I' or 'B'
    weightMod_con = weightMod.addMConstrs(MIP_A, weightMod_x, MIP_sense, MIP_b)
    weightMod.setMObjective(None,MIP_obj,0,sense=gp.GRB.MINIMIZE)
    weightMod.setParam('TimeLimit', run_time)

    weightMod.Params.OutputFlag = 0 # tell gurobi to shut up!!
    weightMod.optimize()
    
    weight_2019 = weightMod_x.x[-100:]
    weight_matrix.append(weight_2019)

    # Calculate 2020 difference
    ret_total_2020 = s2020_stock.values
    diff_2020 = sum(s2020_NDX - ret_total_2020 @ weight_2019)
    diff_2019 = sum(s2019_NDX - ret_total_2019 @ weight_2019)
    
    abs_diff_2019 = sum(abs(s2019_NDX - ret_total_2019 @ weight_2019))
    abs_diff_2020 = sum(abs(s2020_NDX - ret_total_2020 @ weight_2019))
    
    MIP_result_2019.append(diff_2019)
    MIP_result_2020.append(diff_2020)
    
    MIP_abs_2019.append(abs_diff_2019)
    MIP_abs_2020.append(abs_diff_2020)


Academic license - for non-commercial use only - expires 2022-08-25
Using license file C:\Users\dxzha\gurobi.lic
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter TimeLimit to 3600.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter TimeLimit to 3

In [9]:
MIP_dict= {
    'm': m_list,
    '2019_result':MIP_result_2019,
    '2019_abs_result': MIP_abs_2019,
    '2020_result':MIP_result_2020,
    '2020_abs_result':MIP_abs_2020
}

pd.DataFrame(MIP_dict)

,m,2019_result,2019_abs_result,2020_result,2020_abs_result
0,5,-0.097671,0.510542,-0.022066,0.944019
1,10,-0.054992,0.306840,0.029473,0.627855
2,20,-0.034209,0.162467,0.037699,0.523057
3,30,-0.025718,0.113206,0.038013,0.459082
4,40,-0.023372,0.082156,0.023979,0.421344
5,50,-0.028894,0.061218,0.027903,0.387931
6,60,-0.020294,0.052516,0.030244,0.387737
7,70,-0.022825,0.047670,0.024577,0.379463
8,80,-0.022443,0.045227,0.020952,0.370612
9,90,-0.022620,0.044912,0.020086,0.369361


In [11]:
weight_matrix[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.19331871, 0.        , 0.        ,
       0.        , 0.19869909, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.11038484,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.1121602 ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.38543716, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [12]:
weight_matrix[1]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.09277245, 0.11286403, 0.09289557, 0.        ,
       0.        , 0.14516422, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.08710201, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.05444365, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.22678951, 0.        ,
       0.06313178, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [13]:
weight_matrix[2]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.09315366, 0.        , 0.11363998, 0.02968204, 0.03787151,
       0.        , 0.11458236, 0.        , 0.02766421, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.03904179, 0.03631222, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.04671723, 0.        , 0.        , 0.02825467,
       0.        , 0.        , 0.        , 0.04282928, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.12094142, 0.        ,
       0.06553035, 0.        , 0.        , 0.02492895, 0.02190249,
       0.        , 0.        , 0.03376797, 0.05342182, 0.04057

In [14]:
weight_matrix[3]

array([0.        , 0.03704373, 0.        , 0.        , 0.        ,
       0.08782665, 0.        , 0.10371604, 0.01971828, 0.        ,
       0.        , 0.10513349, 0.        , 0.01322143, 0.        ,
       0.03339314, 0.00779714, 0.        , 0.        , 0.01220692,
       0.01432282, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.03186783, 0.        , 0.        ,
       0.03303085, 0.        , 0.02489163, 0.01848805, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.05432351, 0.        , 0.        , 0.03282855,
       0.        , 0.        , 0.        , 0.03151286, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.01404826, 0.11069655, 0.        ,
       0.0430996 , 0.        , 0.        , 0.0149842 , 0.01062526,
       0.        , 0.        , 0.        , 0.        , 0.02720

In [15]:
weight_matrix[4]

array([0.00502745, 0.02640852, 0.00684394, 0.        , 0.        ,
       0.0809014 , 0.        , 0.10361594, 0.01020025, 0.        ,
       0.        , 0.10881146, 0.        , 0.01508076, 0.        ,
       0.04182119, 0.00769982, 0.00689254, 0.        , 0.01187526,
       0.01622568, 0.        , 0.        , 0.        , 0.01089211,
       0.        , 0.        , 0.02738206, 0.01774018, 0.        ,
       0.02798178, 0.        , 0.02165512, 0.02218211, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.05003117, 0.        , 0.        , 0.01962172,
       0.        , 0.00551345, 0.        , 0.03306596, 0.        ,
       0.01397201, 0.        , 0.        , 0.00354096, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.01434863, 0.09878657, 0.        ,
       0.03115604, 0.        , 0.        , 0.02282634, 0.0051423 ,
       0.        , 0.        , 0.        , 0.        , 0.02500

In [16]:
weight_matrix[5]

array([0.00714565, 0.02493645, 0.00399923, 0.        , 0.        ,
       0.        , 0.08133164, 0.09603984, 0.01274117, 0.        ,
       0.        , 0.10030796, 0.01135203, 0.        , 0.        ,
       0.02313697, 0.        , 0.00702073, 0.01039596, 0.00941657,
       0.01464923, 0.0099141 , 0.        , 0.        , 0.0130289 ,
       0.        , 0.00883059, 0.02999946, 0.        , 0.        ,
       0.025267  , 0.        , 0.01634092, 0.01375776, 0.        ,
       0.        , 0.0050091 , 0.01224734, 0.        , 0.        ,
       0.        , 0.05156475, 0.        , 0.00813553, 0.01387246,
       0.        , 0.0096329 , 0.        , 0.02828016, 0.        ,
       0.01134944, 0.00554211, 0.        , 0.0060619 , 0.        ,
       0.        , 0.        , 0.        , 0.01244412, 0.        ,
       0.        , 0.00991884, 0.0077811 , 0.11085054, 0.        ,
       0.01984245, 0.        , 0.        , 0.02060137, 0.00872408,
       0.        , 0.01038931, 0.        , 0.        , 0.01843

In [10]:
weight_matrix[6]

array([0.00389409, 0.02006643, 0.00207782, 0.        , 0.        ,
       0.08796488, 0.        , 0.09466838, 0.01579843, 0.        ,
       0.        , 0.10487747, 0.        , 0.        , 0.        ,
       0.0162243 , 0.00577962, 0.00626309, 0.        , 0.01420853,
       0.01397288, 0.        , 0.        , 0.        , 0.0070053 ,
       0.        , 0.        , 0.02664131, 0.0098835 , 0.        ,
       0.02658581, 0.        , 0.01159798, 0.00495388, 0.        ,
       0.        , 0.0053094 , 0.00706484, 0.00487629, 0.01052201,
       0.        , 0.05258144, 0.        , 0.        , 0.01409301,
       0.        , 0.00616838, 0.00320502, 0.0308515 , 0.00843973,
       0.00907957, 0.        , 0.01030137, 0.00499123, 0.        ,
       0.        , 0.        , 0.00291164, 0.01227466, 0.01512317,
       0.00285318, 0.        , 0.00956348, 0.09648676, 0.        ,
       0.00751806, 0.        , 0.00506293, 0.0164735 , 0.01052008,
       0.00336334, 0.        , 0.00698287, 0.00902252, 0.02298

In [17]:
weight_matrix[7]

array([5.20704585e-03, 1.83910156e-02, 4.63998108e-03, 3.76608232e-03,
       0.00000000e+00, 2.78789478e-02, 5.66729648e-02, 9.95438649e-02,
       1.13032638e-02, 0.00000000e+00, 0.00000000e+00, 1.00039288e-01,
       7.61187541e-03, 0.00000000e+00, 3.84090592e-03, 1.64700961e-02,
       3.94046659e-03, 6.69735848e-03, 6.80435825e-03, 1.14642845e-02,
       1.36250800e-02, 6.36570697e-03, 0.00000000e+00, 0.00000000e+00,
       8.16085931e-03, 0.00000000e+00, 5.42489729e-03, 2.90829455e-02,
       0.00000000e+00, 0.00000000e+00, 2.43783535e-02, 0.00000000e+00,
       1.28964810e-02, 1.22479988e-02, 2.34953752e-03, 0.00000000e+00,
       6.51875760e-03, 7.37806188e-03, 3.11049388e-03, 0.00000000e+00,
       2.70643411e-03, 4.95010505e-02, 2.27166336e-03, 3.94905355e-03,
       1.26798154e-02, 0.00000000e+00, 6.09482050e-03, 0.00000000e+00,
       3.13350719e-02, 6.63532009e-03, 1.24994794e-02, 5.79209983e-03,
       4.21431908e-03, 4.75478667e-03, 4.33680869e-19, 0.00000000e+00,
      

In [18]:
weight_matrix[8]

array([0.00311922, 0.01784337, 0.00356641, 0.00127439, 0.        ,
       0.05713538, 0.02799576, 0.0961838 , 0.01427048, 0.        ,
       0.        , 0.10258284, 0.00406315, 0.00259688, 0.00340856,
       0.01488532, 0.00587308, 0.00679242, 0.00444395, 0.01350555,
       0.0125237 , 0.00430906, 0.        , 0.        , 0.00881589,
       0.        , 0.00226348, 0.02721042, 0.00205537, 0.00336207,
       0.02451171, 0.        , 0.01368896, 0.01049907, 0.00217672,
       0.        , 0.00619408, 0.00654173, 0.00245913, 0.00349148,
       0.00317558, 0.04808425, 0.00230554, 0.00395592, 0.01299778,
       0.        , 0.00634005, 0.00192278, 0.03199645, 0.00770197,
       0.01037404, 0.00472924, 0.00373735, 0.00657555, 0.00332992,
       0.        , 0.00324053, 0.0021276 , 0.00757069, 0.01175325,
       0.00205651, 0.0039064 , 0.00615565, 0.10387313, 0.        ,
       0.01364825, 0.        , 0.00270483, 0.01496808, 0.00953396,
       0.        , 0.00463172, 0.00214238, 0.00731367, 0.01875

In [19]:
weight_matrix[9]

array([0.00423623, 0.01715456, 0.00402742, 0.00246034, 0.        ,
       0.05700554, 0.02962534, 0.09565257, 0.01361514, 0.        ,
       0.        , 0.10201426, 0.00354675, 0.00136721, 0.0038669 ,
       0.01475646, 0.00513451, 0.00666208, 0.00394234, 0.01135048,
       0.01292279, 0.00378376, 0.        , 0.        , 0.00790306,
       0.00168188, 0.00239099, 0.02728529, 0.00240969, 0.00370063,
       0.02321067, 0.        , 0.01121898, 0.01036981, 0.00144418,
       0.        , 0.00685046, 0.00498268, 0.00241398, 0.00404253,
       0.00276433, 0.04732896, 0.00185191, 0.00378876, 0.01211581,
       0.00075119, 0.00537531, 0.00166433, 0.03245533, 0.00758199,
       0.01100542, 0.00515308, 0.00519271, 0.00651449, 0.00240763,
       0.        , 0.00296484, 0.00236596, 0.00909747, 0.01067939,
       0.00210127, 0.00422902, 0.00536573, 0.10112766, 0.00039167,
       0.01397147, 0.00124939, 0.00317284, 0.01628278, 0.00948585,
       0.00229047, 0.00366992, 0.0041711 , 0.00645296, 0.02044